# Running FRAGSYS for SARS-CoV-2 main protease MPro

## Package imports

In [23]:
from fragsys_main import *

## Retrieve superposition data from PDBe GRAPH API

In [2]:
df = pd.read_json("https://www.ebi.ac.uk/pdbe/graph-api/uniprot/superposition/P0DTD1")

In [24]:
df.head(5)

,P0DTD1
0,"{'segment_start': 1, 'segment_end': 180, 'clus..."
1,"{'segment_start': 145, 'segment_end': 180, 'cl..."
2,"{'segment_start': 181, 'segment_end': 818, 'cl..."
3,"{'segment_start': 818, 'segment_end': 929, 'cl..."
4,"{'segment_start': 1023, 'segment_end': 1197, '..."


## Segment 12 of P0DTD1 corresponds to MPro

In [5]:
seg12 = df.loc[11, "P0DTD1"] # segment 12 corresponds to MPro

In [8]:
print(seg12['segment_start'], seg12['segment_end'])

3258 3569


## Segment 12 presents two structural clusters

In [11]:
len(seg12['clusters']) # 2 structural clusters

2

In [18]:
seg12['clusters'][0]

[{'pdb_id': '7vfb',
  'auth_asym_id': 'A',
  'struct_asym_id': 'B',
  'entity_id': 2,
  'is_representative': True},
 {'pdb_id': '7uj9',
  'auth_asym_id': 'C',
  'struct_asym_id': 'B',
  'entity_id': 1,
  'is_representative': False},
 {'pdb_id': '7uj9',
  'auth_asym_id': 'A',
  'struct_asym_id': 'A',
  'entity_id': 1,
  'is_representative': False}]

In [15]:
cluster1_pdbs = list(set([el['pdb_id'] for el in seg12['clusters'][0]]))

In [16]:
cluster2_pdbs = list(set([el['pdb_id'] for el in seg12['clusters'][1]]))

In [17]:
print(len(cluster1_pdbs), len(cluster2_pdbs))

2 719


In [19]:
all_pdbs = sorted(list(set(cluster1_pdbs + cluster2_pdbs)))

## MPro presents 721 unique PDB structures

In [20]:
len(all_pdbs)

721

In [32]:
main_dir = "/cluster/gjb_lab/2394007/pandda_analysis/MPro/"

In [28]:
up_ids = ["MPro" ]* len(all_pdbs)

In [44]:
input_df = pd.DataFrame(list(zip(all_pdbs, up_ids)),
              columns=['pdb_id','entry_uniprot_accession'])

## Generate input dataframe for FRAGSYS

In [125]:
input_df.tail(5)

,pdb_id,entry_uniprot_accession
716,8okl,MPro
717,8okm,MPro
718,8okn,MPro
719,8sk4,MPro
720,8skh,MPro


In [260]:
import importlib
import fragsys_main
importlib.reload(fragsys_main)
from fragsys_main import *

### Remove 43 structures as their assemblies can't currently be downloaded from PDBe

In [216]:
errors = [
    "5sml", "5smm", "5smn", "7sh7", "7sh8",
    "7sh9", "8d4p", "8dib", "8dic", "8did",
    "8die", "8dif", "8dig", "8dih", "8dii",
    "8djj", "8dk8", "8dkh", "8dkk", "8dkl",
    "8dkz", "8dl9", "8dlb", "8dmd", "8dmn",
    "8dzb", "8dzc", "8eir", "8ej7", "8ej9",
    "8eke", "8gfk", "8gfn", "8gfo", "8gfr",
    "8gfu", "8hef", "8okk", "8okl", "8okm",
    "8okn", "8sk4", "8skh"
] # for these 43 structures, biological assemblies could now be downloaded, seems to be a problem with PDBe.

### Remove some other structures as they are to dissimilar in structure/sequence

In [217]:
to_remove = ["7uj9", "7whc", "7uju", "7ujg", "7vfb", "7vfa"]

In [218]:
unconsidered = errors + to_remove

In [220]:
print(len(errors))
print(len(to_remove))
print(len(unconsidered))

43
6
49


In [ ]:
#672 go through ( + 49 = 721 OG structures)

## Run FRAGSYS

In [221]:
input_df = input_df.query('pdb_id not in @unconsidered')

In [ ]:
# Arpeggio crashed for 7m00, 8czx, 7m03, 7m01, 7lzw, 8e68, 7m02, 7m04, 7lzv, 7lzu. This was happening because it could not find neighbouring atoms in the calculation of halogen bond fingerprint.

In [229]:
# Arpeggio crashed for 7tdu, 7n8c, 7lb7. This was happening because the deuterium atoms were not being cleaned out and were not recognised by OpenBabel

/cluster/gjb_lab/2394007/miniconda/envs/arpeggio/bin/python /homes/2394007/SM_VarAlign/arpeggio/arpeggio.py /cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/clean_pdbs/0/7tdu_bio.clean.pdb -s RESNAME:I1W -i 5 -v
    
/cluster/gjb_lab/2394007/miniconda/envs/arpeggio/bin/python /homes/2394007/SM_VarAlign/arpeggio/arpeggio.py /cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/clean_pdbs/0/7n8c_bio.clean.pdb -s RESNAME:YD1 -i 5 -v

/cluster/gjb_lab/2394007/miniconda/envs/arpeggio/bin/python /homes/2394007/SM_VarAlign/arpeggio/arpeggio.py /cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/clean_pdbs/0/7lb7_bio.clean.pdb -s RESNAME:SV6 -i 5 -v

In [239]:
for prot in ["MPro",]:
    print("Starting to process {}".format(prot)) 
    main(main_dir, prot, input_df)

Starting to process MPro
Starting to process group 0 of ['0'] in MPro
Starting CIF to PDB conversion section!
Starting STAMP section!
Starting UNIPROT-PDB mapping section!
Starting ARPEGGIO section!
No LOIs in 7tbt, so skipping!
No LOIs in 7dvy, so skipping!
No LOIs in 7mhn, so skipping!
No LOIs in 5r8t, so skipping!
No LOIs in 8drr, so skipping!
No LOIs in 7vk3, so skipping!
No LOIs in 7mho, so skipping!
No LOIs in 6y2e, so skipping!
No LOIs in 6xoa, so skipping!
No LOIs in 7pxz, so skipping!
No LOIs in 8ds2, so skipping!
No LOIs in 8dry, so skipping!
No LOIs in 7vjy, so skipping!
No LOIs in 7ar5, so skipping!
No LOIs in 7t9y, so skipping!
No LOIs in 7lmc, so skipping!
No LOIs in 7jpy, so skipping!
No LOIs in 8ehm, so skipping!
No LOIs in 7dvw, so skipping!
No LOIs in 7mhh, so skipping!
No LOIs in 7mhf, so skipping!
No LOIs in 8ds1, so skipping!
No LOIs in 7t2t, so skipping!
No LOIs in 7jst, so skipping!


2023-11-29 13:45:09,740 prointvar    INFO     Parsing PDB atoms from lines...


No LOIs in 8d35, so skipping!
No LOIs in 7z2k, so skipping!
No LOIs in 8d4n, so skipping!
No LOIs in 7c2y, so skipping!
No LOIs in 7s82, so skipping!


2023-11-29 13:45:09,949 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 13:45:09,952 prointvar    INFO     PDBx reset atom numbers...
2023-11-29 13:45:10,040 prointvar    INFO     Parsing PDB atoms from lines...


Arpeggio processing failed for 8czx_bio.pdb


...

WARNING!!! -15 residues lacked mapping from PDB to UniProt at arpeggio_lig_cons for 5rey
WARNING!!! -188 residues lacked mapping from PDB to UniProt at lig_cons_split for 5rey
Arpeggio output being processed for 5rey!
Starting BS definition section!


2023-11-29 13:51:11,559 prointvar    INFO     Parsing PDB atoms from lines...


Chimera script successfully created!
Binding sites were sucessfully defined!


2023-11-29 13:51:11,773 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 13:51:11,776 prointvar    INFO     PDBx reset atom numbers...


jackhmmer was generated correctly!
Aln info was correctly created and saved!
There are 0 sequences in the MSA for MPro 0. Skipping to next protein sequence
Fragsys has finished running for MPro!


In [245]:
for prot in ["MPro",]:
    print("Starting to process {}".format(prot)) 
    main(main_dir, prot, input_df)

Starting to process MPro
Starting to process group 0 of ['0'] in MPro
Starting CIF to PDB conversion section!
Starting STAMP section!
Starting UNIPROT-PDB mapping section!
Starting ARPEGGIO section!
No LOIs in 7tbt, so skipping!
No LOIs in 7dvy, so skipping!
No LOIs in 7mhn, so skipping!
No LOIs in 5r8t, so skipping!
No LOIs in 8drr, so skipping!
No LOIs in 7vk3, so skipping!
No LOIs in 7mho, so skipping!
No LOIs in 6y2e, so skipping!
No LOIs in 6xoa, so skipping!
No LOIs in 7pxz, so skipping!
No LOIs in 8ds2, so skipping!
No LOIs in 8dry, so skipping!
No LOIs in 7vjy, so skipping!
No LOIs in 7ar5, so skipping!
No LOIs in 7t9y, so skipping!
No LOIs in 7lmc, so skipping!
No LOIs in 7jpy, so skipping!
No LOIs in 8ehm, so skipping!
No LOIs in 7dvw, so skipping!
No LOIs in 7mhh, so skipping!
No LOIs in 7mhf, so skipping!
No LOIs in 8ds1, so skipping!
No LOIs in 7t2t, so skipping!
No LOIs in 7jst, so skipping!


2023-11-29 15:40:57,935 prointvar    INFO     Parsing PDB atoms from lines...


No LOIs in 8d35, so skipping!
No LOIs in 7z2k, so skipping!
No LOIs in 8d4n, so skipping!
No LOIs in 7c2y, so skipping!
No LOIs in 7s82, so skipping!


2023-11-29 15:40:58,148 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 15:40:58,151 prointvar    INFO     PDBx reset atom numbers...
2023-11-29 15:40:58,324 prointvar    INFO     Parsing PDB atoms from lines...


Arpeggio processing failed for 8czx_bio.pdb
No LOIs in 7nwx, so skipping!


...

2023-11-29 15:41:04,351 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 15:41:04,354 prointvar    INFO     PDBx reset atom numbers...


Arpeggio processing failed for 7lzu_bio.pdb
No LOIs in 7vk0, so skipping!
No LOIs in 7khp, so skipping!
No LOIs in 7uu7, so skipping!
No LOIs in 7dw6, so skipping!
No LOIs in 7tc4, so skipping!
No LOIs in 8ddi, so skipping!
No LOIs in 7uu9, so skipping!
No LOIs in 7vjz, so skipping!
Starting BS definition section!
Shenkin dataframe was created and saved correctly!


2023-11-29 15:41:05,677 py.warnings  WARNING  /cluster/home/2394007/JSU_fragsys/FRAGSYS/fragsys.py:1135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for a given MSA column, calculates some basic statistics

2023-11-29 15:41:06,674 py.warnings  WARNING  /cluster/home/2394007/JSU_fragsys/FRAGSYS/fragsys.py:1136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  such as column residue occupancy ang gaps



Shenkin dataframe was filtered and saved correctly!


NameError: name 'log' is not defined

In [261]:
import importlib
import fragsys_main
importlib.reload(fragsys_main)
from fragsys_main import *

In [262]:
for prot in ["MPro",]:
    print("Starting to process {}".format(prot)) 
    main(main_dir, prot, input_df)

Starting to process MPro
Starting to process group 0 of ['0'] in MPro
Starting CIF to PDB conversion section!
Starting STAMP section!
Starting UNIPROT-PDB mapping section!
Starting ARPEGGIO section!
No LOIs in 7tbt, so skipping!
No LOIs in 7dvy, so skipping!
No LOIs in 7mhn, so skipping!
No LOIs in 5r8t, so skipping!
No LOIs in 8drr, so skipping!
No LOIs in 7vk3, so skipping!
No LOIs in 7mho, so skipping!
No LOIs in 6y2e, so skipping!
No LOIs in 6xoa, so skipping!
No LOIs in 7pxz, so skipping!
No LOIs in 8ds2, so skipping!
No LOIs in 8dry, so skipping!
No LOIs in 7vjy, so skipping!
No LOIs in 7ar5, so skipping!
No LOIs in 7t9y, so skipping!
No LOIs in 7lmc, so skipping!
No LOIs in 7jpy, so skipping!
No LOIs in 8ehm, so skipping!
No LOIs in 7dvw, so skipping!
No LOIs in 7mhh, so skipping!
No LOIs in 7mhf, so skipping!
No LOIs in 8ds1, so skipping!
No LOIs in 7t2t, so skipping!
No LOIs in 7jst, so skipping!


2023-11-29 16:09:37,254 prointvar    INFO     Parsing PDB atoms from lines...


No LOIs in 8d35, so skipping!
No LOIs in 7z2k, so skipping!
No LOIs in 8d4n, so skipping!
No LOIs in 7c2y, so skipping!
No LOIs in 7s82, so skipping!


2023-11-29 16:09:37,463 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 16:09:37,465 prointvar    INFO     PDBx reset atom numbers...
2023-11-29 16:09:37,634 prointvar    INFO     Parsing PDB atoms from lines...


Arpeggio processing failed for 8czx_bio.pdb
No LOIs in 7nwx, so skipping!


...

2023-11-29 16:09:43,089 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 16:09:43,092 prointvar    INFO     PDBx reset atom numbers...
2023-11-29 16:09:43,385 prointvar    INFO     Parsing PDB atoms from lines...


Arpeggio processing failed for 8dru_bio.pdb
No LOIs in 7n5z, so skipping!
No LOIs in 7c2q, so skipping!
No LOIs in 6m03, so skipping!
No LOIs in 7dvp, so skipping!
No LOIs in 7vk7, so skipping!
No LOIs in 8dfn, so skipping!
No LOIs in 7t8m, so skipping!
No LOIs in 6wtm, so skipping!
No LOIs in 7lke, so skipping!
No LOIs in 7kph, so skipping!


2023-11-29 16:09:43,593 prointvar    INFO     PDBx removed existing hydrogens...
2023-11-29 16:09:43,595 prointvar    INFO     PDBx reset atom numbers...


Arpeggio processing failed for 7lzu_bio.pdb
No LOIs in 7vk0, so skipping!
No LOIs in 7khp, so skipping!
No LOIs in 7uu7, so skipping!
No LOIs in 7dw6, so skipping!
No LOIs in 7tc4, so skipping!
No LOIs in 8ddi, so skipping!
No LOIs in 7uu9, so skipping!
No LOIs in 7vjz, so skipping!
Starting BS definition section!
There are 59 sequences in MSA
No human sequences in MSA
['sp|P0DTD1|R1AB_SARS2/3264-3569']
7tbt does not have any LOIs, can't get its ligand binding residues!
7dvy does not have any LOIs, can't get its ligand binding residues!
7mhn does not have any LOIs, can't get its ligand binding residues!
5r8t does not have any LOIs, can't get its ligand binding residues!
8drr does not have any LOIs, can't get its ligand binding residues!
7vk3 does not have any LOIs, can't get its ligand binding residues!
7mho does not have any LOIs, can't get its ligand binding residues!
6y2e does not have any LOIs, can't get its ligand binding residues!
6xoa does not have any LOIs, can't get its ligand

NameError: name 'totals' is not defined

In [241]:
d = pd.read_csv("/cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/results/MPro_0_BS_def_OC_complete_i_rel_0.66.csv")

In [250]:
d = pd.read_pickle("/cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/results/varalign/0/MPro_0_rf.sto_mappings.p.gz")

In [254]:
aln_ids = list(set([seqid[0] for seqid in d.index.tolist() if "P0DTD1" in seqid[0]])) # THIS IS EMPTY IF QUERY SEQUENCE IS NOT FOUND

In [255]:
aln_ids

['sp|P0DTD1|R1AB_SARS2/3264-3569']

In [258]:
dy = pd.read_csv("/cluster/gjb_lab/2394007/pandda_analysis/MPro/MPro/results/MPro_0_fragsys_df.csv")

In [259]:
dy

,index,UniProt_ResNum,Pfam_column,alignment_column,shenkin,occ,gaps,occ_pct,gaps_pct,rel_norm_shenkin,...,BS95,BS96,BS97,BS98,BS99,BS100,BS101,BS102,BS103,BS104
0,4,3266,3,3,17.835177,59,0,1.000000,0.000000,31.507456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6,3267,4,4,17.925256,59,0,1.000000,0.000000,31.747262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,3268,5,5,6.957371,59,0,1.000000,0.000000,2.548702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10,3269,6,6,8.335726,59,0,1.000000,0.000000,6.218141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,3270,7,7,15.514397,59,0,1.000000,0.000000,25.329106,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,14,3271,8,8,33.064855,59,0,1.000000,0.000000,72.051707,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,254,3386,127,127,30.372135,59,0,1.000000,0.000000,64.883183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,258,3387,128,128,7.335771,59,0,1.000000,0.000000,3.556072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,270,3390,131,131,29.716529,59,0,1.000000,0.000000,63.137836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,322,3415,156,156,23.615656,59,0,1.000000,0.000000,46.896170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


NameError: name 'arpeggio_dir' is not defined